## The advent of Big Data

**90%** of the data in the world today has been created in the last two years alone. Our current output of data is roughly 2.5 quintillion bytes a day. As the world steadily becomes more connected with an ever-increasing number of electronic devices, that’s only set to grow over the coming years. Now our local computers are not capable of loading and processing such huge chunks of data. I remember loading a 1GB file once into my local machine for analysis, and my machine was very slow in processing it(poor i3).The problem is that **Pandas** loads the entire dataset into the RAM. Thus processing large datasets takes time. Also we cannot load complete databases into Pandas.

The solution to this problem is **SQL**. We can use SQL to transform the data into something manageable by pandas before loading it in memory. SQL is very efficient slicing and dicing the data, but lacks the power of analytics as that in Pandas. To overcome this analytics incapability, Google has come up with its great product **BIGQUERY**.

**Google BigQuery** is a cloud-based big data analytics web service for processing very large read-only data sets.
BigQuery was designed for analyzing data on the order of billions of rows, using a SQL-like syntax. It runs on the Google Cloud Storage infrastructure and can be accessed with a REST-oriented application program interface (API). Now we cannot directly query BigTable using Pandas. So we need to first extract the desired data from the pool of data available, convert it into Pandas Dataframe and then explore and analyse it. BigQuery has support for SQL syntax, thus we will directly query and extract data from the BigTable using SQL and carry forward analysis using Pandas. 

This transition of BigTable data into Pandas Dataframe is very easy and is just 2-3 steps procedure. I would highly encourage you to please this **[BigQuery Starter Notebook by Megan Risdal](https://www.kaggle.com/mrisdal/safely-analyzing-github-projects-popular-licenses)**. Thanks a lot Megan for the notebook:)

Now coming to the Notebook, I will try to explore the Eclipse MegaMovie Dataset, which is a collection of photos submitted by a group of volunteers from locations across the United States during the August 21, 2017 total solar eclipse. I will also try to explain the SQL queries wherever needed.

##### If you like this notebook, Do upvote it. Thanks..!!

In [ ]:
# Loading packages for analytics work
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
plt.style.use('fivethirtyeight')
import folium
import folium.plugins
from folium import IFrame
from matplotlib import animation,rc
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')
import io
from scipy.misc import imread
import codecs
import base64
from wordcloud import WordCloud, STOPWORDS

In [ ]:
# Loading packages for BigQuery work
from bq_helper import BigQueryHelper
bq_assistant = BigQueryHelper("bigquery-public-data", "eclipse_megamovie")

#### Lets first check what the columns of the dataset contain: 

**id:** Unique identifier for each photo.

**upload_session_id:** Identifier that joins all photos uploaded in a single session by a single user

**user:** A numeric ID which is unique for each user

**image_bucket:** The bucket the image was assigned to- 'megamovie' if the uploader was a member of the Megamovie Photo Team.

**storage_uri:** The URI at which the photo can be downloaded from Google Cloud Storage

**model:** The model of the camera

**make:** Company of the camera

**uploaded_date:** The date and time at which the photo was uploaded to the Megamovie website

**camera_datetime:** The time at which the photo was taken, according to the Camera clock

**image_datetime:** The date and time at which the photo was taken, as provided by GPS data or imputed from the camera_datetime

**datetime_repaired:** True if the image_datetime field was imputed from the camera_datetime and the timezone adjusted by the lat/lon

**lat:** Latitude at which photo was taken

**equatorial_mount:** Whether the camera was mounted equatorially (true) or not (false)

**lon:** Longitude at which the photo was taken

**State:** the US state that the photo was taken in

**image_type:** The file format of the image

**width:** Width of photo, in pixels

**height:** Height of the photo, in pixels

**detected_circle:**.radius - the radius of the detected circle (in pixels), .cx - the x coordinate center of the detected circle (in pixels), .cy - the y coordinate center of the detected circle (in pixels)
exposure time: The exposure time of the photograph, represented as a ratio

**vision_labels:** list of labels predicted for the image by Google Cloud Vision

**is_mobile:** whether the photo was taken on a mobile phone

**totality:** whether the photo is predicted to be in totality by its GPS lat/lon/image_datetime

### Firing the Simplest Query (checking total rows in the file)

In [ ]:
Query="""
Select count(*) as count
From `bigquery-public-data.eclipse_megamovie.photos_v_0_1`"""

size=bq_assistant.query_to_pandas_safe(Query)

print('The dataset1 has record for ',size['count'].values[0],'photos')

So the first file has 35294 rows. But we have 3 files in the dataset. So let us check total rows in all the 3 files. A simple but tedious way is running the same query with different file and adding the output of all 3 queries. But for a lazy person like me, lets fire multiple queries in a single go.

**Note: Always remember the proper sequence of a SQL Query**
 - FROM

 - ON

 - WHERE

 - GROUP BY

 - HAVING

 - SELECT

 - DISTINCT

 - ORDER BY

In [ ]:
Query="""
SELECT(Select count(*) 
From `bigquery-public-data.eclipse_megamovie.photos_v_0_1`)
+
(Select count(*) 
From `bigquery-public-data.eclipse_megamovie.photos_v_0_2`)
+
(Select count(*) 
From `bigquery-public-data.eclipse_megamovie.photos_v_0_3`
) as sum_photos
"""

size = bq_assistant.query_to_pandas_safe(Query)
print('The complete dataset has record for ',size['sum_photos'].values[0],'photos')

So the above query is a type of Nested Query or Subquery which is very common in SQL. It is nothing but a query inside another query. A subquery can appear in a **SELECT, FROM or WHERE** statement. Above we have it in the Select Statement.

The query works by first executing each of the 3 subqueries and then the output of each of the subqueries are added and the final sum is calculated.

### Checking number of users in the dataset

We are not sure if a user's information is just limited to a single file i.e we do not know if a user eg:Alex's information is just present in file 1 or also in file 2 and 3. So we need to first append or concat all the 3 files and then find the distinct users in them. We can do this in a few ways. We will see 2 such queries and also check the time both of them take to execute. The query with the least time should be preferred, as **Time is Money**...xd

#### Using UNION DISTINCT

In [ ]:
%%time
Query="""
Select user
From `bigquery-public-data.eclipse_megamovie.photos_v_0_1`

UNION DISTINCT

Select user
From `bigquery-public-data.eclipse_megamovie.photos_v_0_2`

UNION DISTINCT

Select user
From `bigquery-public-data.eclipse_megamovie.photos_v_0_3`
"""
n_users = bq_assistant.query_to_pandas_safe(Query)
print('The total number of unique users in the dataset are:',n_users.shape[0])


#### Using UNION ALL

In [ ]:
%%time
Query="""
Select Count(Distinct x.user)
From(
     Select user
     From `bigquery-public-data.eclipse_megamovie.photos_v_0_1`
     
     UNION ALL
     
     Select user
     From `bigquery-public-data.eclipse_megamovie.photos_v_0_2`
     
     UNION ALL
     
     Select user
     From `bigquery-public-data.eclipse_megamovie.photos_v_0_3`) x
Group By x.user
"""
n_user = bq_assistant.query_to_pandas_safe(Query)
print('Number of Distinct Users:',n_user.shape[0])

It is evident that **UNION ALL** takes less time than **UNION DISTINCT**. Lets see what the query does.

**UNION DISTINCT:** It extracts the user column from all the 3 files. Before appending two outputs, it checks for distinct users. This operation is done twice and thus we have a list of all unique users.

**UNION ALL:** It extracts the user column from all the 3 files, and directly appends all the 3 files. It then group's by the users and finally the count of unique users is given.

Lets see average photos takn by each user

In [ ]:
import math
print('Average Photos taken by Each User:',math.ceil(size['sum_photos'].values[0]/n_users.shape[0]))

### Unique Devices in the dataset

In [ ]:
Query="""
Select Count(Distinct x.model) as distinct_models
From(
     Select model
     From `bigquery-public-data.eclipse_megamovie.photos_v_0_1`
     
     UNION ALL
     
     Select model
     From `bigquery-public-data.eclipse_megamovie.photos_v_0_2`
     
     UNION ALL
     
     Select model
     From `bigquery-public-data.eclipse_megamovie.photos_v_0_3`) x
Group By x.model
"""
n_model = bq_assistant.query_to_pandas_safe(Query)
print('Number of Distinct Camera Devices are:',n_model.shape[0])

### Most Used Camera Devices

In [ ]:
Query1="""
Select model,count(Distinct user) as Count
From (
      Select model,user 
      From `bigquery-public-data.eclipse_megamovie.photos_v_0_1`
      
      UNION ALL
      
      Select model,user 
      From `bigquery-public-data.eclipse_megamovie.photos_v_0_2`
      
      UNION ALL
      
      Select model,user 
      From `bigquery-public-data.eclipse_megamovie.photos_v_0_3`
      )
Group By model
Order By Count DESC
"""
model_count = bq_assistant.query_to_pandas_safe(Query1)
ax=model_count[1:16].plot(x='model',y='Count',kind='barh',width=0.9,color=sns.color_palette('winter_r',15))
for i, v in enumerate(model_count[1:16].Count.values): 
    ax.text(.8, i, v,fontsize=12,color='white',weight='bold')
plt.gca().invert_yaxis()
ax.patches[0].set_facecolor('r')
fig=plt.gcf()
fig.set_size_inches(8,10)
plt.title('Most Used Camera Devices')
plt.show()


Wow!! Iphone 6 is the highest used device for capturing photos, also higher as compared to DSLR's. We can see that Iphones are competing well with the DSLR's. A prime reason for this is surely the compact size and its great camera quality. 

Coming to the query, we first extract the user and model columns from the files, then groupby the data extracted using the models and get a count of users using the devices. We then sort the data using the Count column in descending order.

Now let's check the most used camera's by the is_mobile column i.e lets check the most famous mobile camera and DSLR camera companies.

In [ ]:
Query1="""
Select make,count(Distinct user) as Count
From (
      Select make,user
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_1`
      Where is_mobile=True
      UNION ALL
      Select make,user
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_2`
      Where is_mobile=True
      UNION ALL
      Select make,user
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_3`
      Where is_mobile=True)
Group By make
Order By Count DESC
"""
makemob_count=bq_assistant.query_to_pandas_safe(Query1)
Query2="""
Select make,count(Distinct user) as Count
From (
      Select make,user
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_1`
      Where is_mobile=False
      UNION ALL
      Select make,user
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_2`
      Where is_mobile=False
      UNION ALL
      Select make,user
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_3`
      Where is_mobile=False)
Group By make
Order By Count DESC
"""
makecam_count=bq_assistant.query_to_pandas_safe(Query2)


Above we use 2 seperate queries to get two separate dataframes for top mobile cameras and DSLR's. The only difference between the above and previous query is that we have added a **WHERE** statement, which is used to filter out mobiles and DSLR's using the **is_mobile=True/False** condition.


In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
ax1=makemob_count[:5].plot(x='make',y='Count',kind='barh',width=0.8,ax=ax[0],color=sns.color_palette('inferno_r',15))
ax[0].invert_yaxis()
ax1.patches[0].set_facecolor('r')
for i, v in enumerate(makemob_count[:5].Count.values): 
    ax1.text(.8, i, v,fontsize=14,color='b',weight='bold')
ax[0].set_title('Top Mobile Camera Makers')
ax2=makecam_count[:5].plot(x='make',y='Count',kind='barh',width=0.8,ax=ax[1],color=sns.color_palette('inferno_r',15))
ax[1].invert_yaxis()
ax2.patches[0].set_facecolor('r')
for i, v in enumerate(makecam_count[:5].Count.values): 
    ax2.text(.8, i, v,fontsize=14,color='b',weight='bold')
ax[1].set_title('Top DSLR Makers')
plt.subplots_adjust(wspace=0.8)

### Camera Aperature

In [ ]:
Query="""
Select make,aperture_value
From (
      Select make,aperture_value
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_1`
      Where is_mobile=False
      UNION ALL
      Select make,aperture_value
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_2`
      Where is_mobile=False
      UNION ALL
      Select make,aperture_value
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_3`
      Where is_mobile=False
      )
"""
Query1="""
Select make,aperture_value
From (
      Select make,aperture_value
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_1`
      Where is_mobile=True
      UNION ALL
      Select make,aperture_value
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_2`
      Where is_mobile=True
      UNION ALL
      Select make,aperture_value
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_3`
      Where is_mobile=True
      )
"""
aper_cam=bq_assistant.query_to_pandas_safe(Query)
aper_mob=bq_assistant.query_to_pandas_safe(Query1)
aper_cam=aper_cam.dropna()
aper_mob=aper_mob.dropna()

In [ ]:
f,ax=plt.subplots(2,1,figsize=(22,14))
sns.boxplot('make','aperture_value',data=aper_cam[aper_cam['make'].isin(makecam_count['make'][:10])],ax=ax[0])
ax[0].set_title('Aperatures for DSLR')
sns.boxplot('make','aperture_value',data=aper_mob[aper_mob['make'].isin(makemob_count['make'][:10])],ax=ax[1])
ax[1].set_title('Aperatures for Mobile Camera')

It is evident that the aperature's of the professional DSLR's have a very high range as compared to mobile camera's. Some DSLR's aperture are even more than 40, whereas the range of aperature for mobile phones is generally between 2 and 3, with a few exceptions going above 5.

### Labels

Let's check the labels or keywords that the Computer Vision Algorithm from Google generated by processing the image.

In [ ]:
Query="""
Select vision_labels 
From `bigquery-public-data.eclipse_megamovie.photos_v_0_1`
"""
labels=bq_assistant.query_to_pandas_safe(Query)
list1=[]
for i in labels[labels['vision_labels']!=''].vision_labels.str.split(','):
    list1.extend(i)
labels=pd.Series(list1)

In [ ]:
insta=b'...'

In [ ]:
f1=open("insta.png", "wb")
f1.write(codecs.decode(insta,'base64'))
f1.close()
img1 = imread("insta.png")
hcmask1 = img1
wc = WordCloud(background_color="black", max_words=1000, mask=hcmask1, 
               stopwords=STOPWORDS, max_font_size= 60,width=1000,height=1000)
wc.generate(" ".join(labels))
plt.imshow(wc)
plt.axis('off')
fig=plt.gcf()
fig.set_size_inches(10,10)
plt.show()

As expected, the algorithm generated keywords related to celestial bodies,eclipse, colur of the sky, etc.

## User with Highest Clicks

In [ ]:
Query1="""
Select lon,lat,user,model,storage_uri,state,is_mobile,height,width
From (
      Select lon,lat,user,model,storage_uri,state,is_mobile,height,width
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_1`
      UNION ALL
      Select lon,lat,user,model,storage_uri,state,is_mobile,height,width
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_2`
      UNION ALL
      Select lon,lat,user,model,storage_uri,state,is_mobile,height,width
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_3`
      )
"""
user_loc=bq_assistant.query_to_pandas_safe(Query1)
user_loc.dropna(inplace=True)

In [ ]:
Query="""
SELECT user, COUNT(user) 
FROM (
Select user
      From `bigquery-public-data.eclipse_megamovie.photos_v_0_1`
      UNION ALL
      Select user
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_2`
      UNION ALL
      Select user
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_3`
) 
GROUP BY user 
HAVING COUNT (user)=( 
SELECT MAX(mycount) 
FROM ( 
SELECT user, COUNT(user) mycount 
FROM (
Select user
      From `bigquery-public-data.eclipse_megamovie.photos_v_0_1`
      UNION ALL
      Select user
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_2`
      UNION ALL
      Select user
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_3`
)
GROUP BY user))
"""
max_user=bq_assistant.query_to_pandas_safe(Query)

In [ ]:
max_user=user_loc[user_loc['user'].isin(max_user['user'])]
print('Maximum Photos was taken by user',max_user['user'].unique()[0],'with',max_user.shape[0],'photos')

### Camera's used By the Top Photographer

In [ ]:
ax=max_user['model'].value_counts().plot.barh(width=0.8)
for i, v in enumerate(max_user['model'].value_counts().values): 
    ax.text(.8, i, v,fontsize=12,color='black',weight='bold')
plt.title('Cameras used by the Photographer')
plt.gca().invert_yaxis()

## Mapping All The Users (Dataset Inspiration)

As stated in the dataset, the inspiration of the dataset is to map the locations of the project contributors i.e the photographers. We will using a Folium Map for doing so. We will locate each photographer on the map and check how many were in and out of the **Path of Totality**.

### Path Of Totality

The path of totality in layman terms is the line along which the solar eclipse motion is followed. It is mostly a straight line, but it looks curved due to the earth's curved surface.

In [ ]:
user_loc=user_loc.drop_duplicates(subset=['user'])
locate=user_loc[['lat','lon']]
model=user_loc['model']
state=user_loc['state']
height=user_loc['height']
width=user_loc['width']
img=user_loc['storage_uri']
mob_cam=user_loc['is_mobile']
def color(value):
    if value==True:
        return 'green'
    else:
        return 'red'

In [ ]:
map1 = folium.Map(location=[39.50, -98.35],tiles='CartoDB dark_matter',zoom_start=3.5)
for point in user_loc.index:
    info='<b>Model:</b>'+str(model[point])+'<br><b>State</b>: '+str(state[point])+'<br><b>Img Height</b>: '+str(height[point])+'px'+'<br><b>Img Width</b>: '+str(width[point])+'px'+'<br><b>Img Src</b>: '+str(img[point])
    iframe = folium.IFrame(html=info, width=250, height=250)
    folium.CircleMarker(list(locate.loc[point].values),popup=folium.Popup(iframe),radius=1,color=color(mob_cam[point])).add_to(map1)
map1

The above map plots the users by their location of photographers. The green points are for users who clicked photos using their cellphones, whereas the red points are those who clicked photos using professional camera's.

We can clearly see the **Path of Totality**, from the north-west part of USA to south-eastern part. As far as I know the USA map, the states of Wyoming, Oregon, Georgia, South Carolina, Illinois, and many more fall under the path of totality. 

Almost all photgraphers were in the path os totality, sparring a few who were at the south-western and north-eastern parts of USA. And also a single user from UK maybe....lol!!

So we have located all the contributors of the project, and can see how many of them were in and away from the path of totallity
Click on the points to get additional informtion. Copy the link in another tab from the popup to see the image.

## Mapping Photos Taken By Hours

In [ ]:
Query="""
Select lat,lon,EXTRACT(HOUR from image_datetime) as hour
FROM (
Select lat,lon,image_datetime
      From `bigquery-public-data.eclipse_megamovie.photos_v_0_1`
      where image_datetime Is Not Null and lat is not Null and lon is not Null
      UNION ALL
      Select lat,lon,image_datetime
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_2`
      where image_datetime Is Not Null and lat is not Null and lon is not Null
      UNION ALL
      Select lat,lon,image_datetime
      From`bigquery-public-data.eclipse_megamovie.photos_v_0_3`
      where image_datetime Is Not Null and lat is not Null and lon is not Null
) """
rel=bq_assistant.query_to_pandas_safe(Query)

In [ ]:
fig=plt.figure(figsize=(10,8))
from mpl_toolkits.basemap import Basemap
from matplotlib import animation,rc
fig.text(.3, .1, '21 August 2017', ha='right')
def animate(Hour):
    ax = plt.axes()
    ax.clear()
    ax.set_title('Photos Clicked '+'\n'+'Hour:' +str(Hour))
    m6 = Basemap(llcrnrlon=-119,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=49,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
    lat_gif1=list(rel[rel['hour']==Hour].lat)
    long_gif1=list(rel[rel['hour']==Hour].lon)
    x_gif1,y_gif1=m6(long_gif1,lat_gif1)
    m6.scatter(x_gif1, y_gif1,color ='r') 
    m6.drawcoastlines()
    m6.drawcountries()
    m6.fillcontinents(color='coral',lake_color='aqua', zorder = 1,alpha=0.4)
    m6.drawmapboundary(fill_color='aqua')
ani = animation.FuncAnimation(fig,animate,list(rel.hour.unique()), interval = 1500)    
ani.save('animation.gif', writer='imagemagick', fps=1)
plt.close(1)
filename = 'animation.gif'
video = io.open(filename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))

The above animation shows the loactions of photos taken by hours. It is evident that the maximum number of photos were taken in the hours -17 and 18.

I hope this notebook proved to be a good source for getting started with BigQuery. If u liked the notebook **Do Upvote**.

## Thank You